In [ ]:
# Import Libraries
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import cv2
import random
import os
import numpy as np
import pandas as pd
import matplotlib.patches as patches
from keras.preprocessing.image import ImageDataGenerator,load_img
import random
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Activation,BatchNormalization

In [ ]:
# Data Exploration - Whats in the box csv file?
day_clip6_anno_path_box = "./data/sample-dayClip6/sample-dayClip6/frameAnnotationsBOX.csv"

pd.read_csv(day_clip6_anno_path_box, sep=';') 

In [ ]:
# Data Exploration - Whats the difference between the BOX and BULB files?
day_clip6_anno_path_bulb = "./data/sample-dayClip6/sample-dayClip6/frameAnnotationsBULB.csv"

pd.read_csv(day_clip6_anno_path_bulb, sep=';') 

In [ ]:
# What variety of tags do we have for our model?

dayTrainFolder = "./data/Annotations/Annotations/dayTrain/"
annotation_folders = os.listdir(dayTrainFolder)

for folder in annotation_folders:
    annotation_file = dayTrainFolder+folder+"/frameAnnotationsBOX.csv"
    print('-----From folder:', folder)
    print((pd.read_csv(annotation_file, sep=';'))['Annotation tag'].value_counts())


In [ ]:
# I would like to know the total annotations found in each folder. This is relavant for when we train our model.

dayTrainFolder = "./data/Annotations/Annotations/dayTrain/"
annotation_folders = os.listdir(dayTrainFolder)

annotation_totals = {'stop':0, 'go':0, 'stopLeft':0, 'goLeft':0, 'warningLeft':0, 'warning':0}

for folder in annotation_folders:
    annotation_file = dayTrainFolder+folder+"/frameAnnotationsBOX.csv"
    #print((pd.read_csv(annotation_file, sep=';'))['Annotation tag'].value_counts())
    item = (pd.read_csv(annotation_file, sep=';'))['Annotation tag'].value_counts()
    try:
        if item.stop:
            annotation_totals['stop'] += item.stop
    except:
        pass
    try:
        if item.go:
            annotation_totals['go'] += item.go
    except:
        pass
    try:
        if item.stopLeft:
            annotation_totals['stopLeft'] += item.stopLeft
    except:
        pass
    try:
        if item.goLeft:
            annotation_totals['goLeft'] += item.goLeft
    except:
        pass
    try:
        if item.warningLeft:
            annotation_totals['warningLeft'] += item.warningLeft
    except:
        pass
    try:
        if item.warning:
            annotation_totals['warning'] += item.warning
    except:
        pass

print(annotation_totals)
 

In [ ]:
#Let's see it in a graph
plt.bar(range(len(annotation_totals)), list(annotation_totals.values()), align='center')
plt.xticks(range(len(annotation_totals)), list(annotation_totals.keys()))

print("Total annotations found by category:")
plt.show()

# I am starting to think I wont have enough images in these categories to sufficiently train the model accross them all

In [ ]:
# Bring it all together... This takes about 40 minutes, sooo...dont run it unless you have to. 
# or at least remove some of the frames folders

frame_path_arr = []
annotations = []
images = []
labels = []
color_space = [(0,255,0),(255,0,0),(255,0,0)]

def process_images():
    for folder in os.listdir("./data/dayTrain/dayTrain/"):
        subfolder = os.listdir("./data/dayTrain/dayTrain/"+folder+"/"+"/frames")
        
        for file in subfolder:
            frame_path_arr.append("./data/dayTrain/dayTrain/"+folder+"/"+"/frames/"+file)

    for img_path in frame_path_arr:
        frame_name = img_path[::-1][0:img_path[::-1].index("/")][::-1]
        #print(frame_name)
        frame_anno = "./data/Annotations/Annotations/dayTrain"+"/"+img_path[25:34].replace("/","")+"/frameAnnotationsBOX.csv"
        #print(frame_anno)
        
        with open(frame_anno) as fa:  
            line = fa.readline()
            line = fa.readline()
            while line:
                anno_file_path = (line.strip()).split(";")
                anno_file_id = anno_file_path[0].split("/")[1]
                #print(anno_file_path, anno_file_id)
                #print(anno_file_id, frame_name)
                if anno_file_id == frame_name:
                    annotations.append(anno_file_path)
                line = fa.readline()

    for img_path in frame_path_arr:
        for anno in annotations:
            #print(img_path[::-1][0:img_path[::-1].index("/")][::-1])
            #print(str(anno).split(",")[0].replace("'", "").replace("[", "").replace("dayTraining/", ""))
            if img_path[::-1][0:img_path[::-1].index("/")][::-1] == str(anno).split(",")[0].replace("'", "").replace("[", "").replace("dayTraining/", ""):
                category = anno[1]
                left = int(anno[2])
                top = int(anno[3])
                right = int(anno[4])
                bottom = int(anno[5])
                #print("file:", img_path, "\tCategory: '{}' at [{},{},{},{}]".format(category, left, top, right, bottom))
                img = cv2.imread(img_path)
                #img = cv2.resize(img, (25, 25))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                #plt.rcParams['figure.figsize'] = [32, 16]
                cropped_image = img[top:bottom, left:right]
                cropped_image = cv2.resize(cropped_image, (25, 25))
                
                try:
                    if category == "stop":
                        labels.append(0)
                except:
                    pass
                try:
                    if category == "go":
                        labels.append(1)
                except:
                    pass
                try:
                    if category == "stopLeft":
                        labels.append(0)
                except:
                    pass
                try:
                    if category == "goLeft":
                        labels.append(1)
                except:
                    pass
                try:
                    if category == "warningLeft":
                        labels.append(2)
                except:
                    pass
                try:
                    if category == "warning":
                        labels.append(2)
                except:
                    pass
                
                images.append(cropped_image)
                print("Working on: ", img_path[::-1][0:img_path[::-1].index("/")][::-1])
                # --- uncomment to show images ---
                cv2.imshow("cropped", cropped_image)
                plt.imshow(cropped_image)
                plt.show()   
                

process_images()

data = np.array(images)
labels = np.array(labels)

In [ ]:
#Total images and labels:
print("Images: ", len(images))
print("Labels: ", len(labels))

data = np.array(data)
labels = np.array(labels)
print("Shapes pre-split", data.shape, labels.shape)


X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
print("Shapes post-split", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

#print(train_test_split(data, labels, test_size=0.2, random_state=42))
#print(y_train)
#print(y_test)

In [ ]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# I think 15 epochs is going to over fit it... lets see though

history = model.fit(X_train, y_train, batch_size=6000, epochs=15, validation_data=(X_test, y_test))
model.save("C964_TrafficLightModel.h5")

In [ ]:
#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Lets test the model with images its never seen before

# Lets look at our results

test_files=os.listdir("./data/MyTestImages")

CATEGORIES = ["Red", "Green"]

model = tf.keras.models.load_model("C964_TrafficLightModel.h5")

accuracy_total = {'Green':0, 'Red':0}
total_red = 0
total_yellow = 0
total_green = 0
total_red_correct_predict = 0
total_yellow_correct_predict = 0
total_green_correct_predict = 0


def prep_img(file):
    img = cv2.imread(file)
    new_img = cv2.resize(img, (25, 25))
    final_img = new_img.reshape(1, 25, 25, 3)
    #plt.imshow(final_img)
    #plt.show()
    #print(new_img.shape)
    return final_img


index = 0
for file in test_files:
    prediction = model.predict(prep_img("./data/MyTestImages/"+file))
    #print(prediction)
    print("File", file, "has been predicted as being: ",
         
         "{:.2f}".format(prediction[0][0]*100), "% Red - ",
          "{:.2f}".format(prediction[0][1]*100), "% Green - "
          #,"{:.2f}".format(prediction[0][2]*100), " % Yellow"
         
         
         
         )
    
    if file.find("RED") == 0:
        total_red += 1
        if "{:.2f}".format(prediction[0][0]*100) > ("{:.2f}".format(prediction[0][1]*100)):
            total_red_correct_predict += 1

    if file.find("GREEN") == 0:
        total_green += 1
        if "{:.2f}".format(prediction[0][1]*100) > ("{:.2f}".format(prediction[0][0]*100)):
            total_green_correct_predict += 1
        
    #if file.find("yellow"):
        #total_yellow += 1
        #if "{:.2f}".format(prediction[0][2]*100) > ("{:.2f}".format(prediction[0][1]*100) + "{:.2f}".format(prediction[0][1]*100)):
            #total_yellow_correct_predict += 1


    
print("Red Accuracy:", (total_red_correct_predict/total_red)*100,"%")
print("Green Accuracy:", (total_green_correct_predict/total_green)*100,"%")
#print("Yellow Accuracy:" , (total_yellow_correct_predict/total_yellow)*100)

accuracy_total['Red'] += (total_red_correct_predict/total_red)*100
accuracy_total['Green'] += (total_green_correct_predict/total_green)*100
#accuracy_total['Yellow'] += (total_yellow_correct_predict/total_yellow)*100


#Let's see it in a graph
plt.bar(range(len(accuracy_total)), list(accuracy_total.values()), align='center')
plt.xticks(range(len(accuracy_total)), list(accuracy_total.keys()))

print("\nAccuracy By Light Color:")
plt.show()

In [ ]:
# Bring it all together... This takes about 40 minutes, sooo...dont run it unless you have to. 
# or at least remove some of the frames folders
# This one is a bit different as it will only allow a max of 14681 red images and 14681 green images

frame_path_arr = []
annotations = []
images = []
labels = []
color_space = [(0,255,0),(255,0,0),(255,0,0)]

def process_images():
    CUR_GR_IMG = 0
    CUR_RE_IMG = 0
    CUR_YE_IMG = 0
    IMG_MAX = 14681

    for folder in os.listdir("./data/dayTrain/dayTrain/"):
        subfolder = os.listdir("./data/dayTrain/dayTrain/"+folder+"/"+"/frames")
        
        for file in subfolder:
            frame_path_arr.append("./data/dayTrain/dayTrain/"+folder+"/"+"/frames/"+file)

    for img_path in frame_path_arr:
        frame_name = img_path[::-1][0:img_path[::-1].index("/")][::-1]
        #print(frame_name)
        frame_anno = "./data/Annotations/Annotations/dayTrain"+"/"+img_path[25:34].replace("/","")+"/frameAnnotationsBOX.csv"
        #print(frame_anno)
        
        with open(frame_anno) as fa:  
            line = fa.readline()
            line = fa.readline()
            while line:
                anno_file_path = (line.strip()).split(";")
                anno_file_id = anno_file_path[0].split("/")[1]
                #print(anno_file_path, anno_file_id)
                #print(anno_file_id, frame_name)
                if anno_file_id == frame_name:
                    annotations.append(anno_file_path)
                line = fa.readline()

    for img_path in frame_path_arr:
        for anno in annotations:
            
            if CUR_RE_IMG >= IMG_MAX  and CUR_GR_IMG >= IMG_MAX:
                break
                
            try:
                if category == "warningLeft":
                    break
            except:
                pass
            try:
                if category == "warning":
                    break
            except:
                pass
            
            
            #print(img_path[::-1][0:img_path[::-1].index("/")][::-1])
            #print(str(anno).split(",")[0].replace("'", "").replace("[", "").replace("dayTraining/", ""))
            if img_path[::-1][0:img_path[::-1].index("/")][::-1] == str(anno).split(",")[0].replace("'", "").replace("[", "").replace("dayTraining/", ""):
                category = anno[1]
                left = int(anno[2])
                top = int(anno[3])
                right = int(anno[4])
                bottom = int(anno[5])
                #print("file:", img_path, "\tCategory: '{}' at [{},{},{},{}]".format(category, left, top, right, bottom))
                img = cv2.imread(img_path)
                #img = cv2.resize(img, (25, 25))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                #plt.rcParams['figure.figsize'] = [32, 16]
                cropped_image = img[top:bottom, left:right]
                cropped_image = cv2.resize(cropped_image, (25, 25))
                
                try:
                    if category == "stop" and CUR_RE_IMG < IMG_MAX:
                        images.append(cropped_image)
                        labels.append(0)
                        CUR_RE_IMG += 1
                except:
                    pass
                try:
                    if category == "go" and CUR_GR_IMG < IMG_MAX:
                        images.append(cropped_image)
                        labels.append(1)
                        CUR_GR_IMG += 1
                except:
                    pass
                try:
                    if category == "stopLeft" and CUR_RE_IMG < IMG_MAX:
                        images.append(cropped_image)
                        labels.append(0)
                        CUR_RE_IMG += 1
                except:
                    pass
                try:
                    if category == "goLeft" and CUR_GR_IMG < IMG_MAX:
                        images.append(cropped_image)
                        labels.append(1)
                        CUR_GR_IMG += 1
                except:
                    pass


                print("Working on: ", img_path[::-1][0:img_path[::-1].index("/")][::-1])
                # --- uncomment to show images ---
                #cv2.imshow("cropped", cropped_image)
                #plt.imshow(cropped_image)
                #plt.show()   
                

process_images()

data = np.array(images)
labels = np.array(labels)